# Camp Gladiator Location Search

This tool parses the training locations feed to give us a searchable data set

In [97]:
import pandas as pd
import requests, json
from copy import copy
from pandas.io.json import json_normalize

Download the data and parse it using the Python `json` library

In [47]:
url = "https://campgladiator.com/locations/get_locations?use_bounds=1&b_ne=(30.375762407897696%2C%20-97.3874050279785)&b_sw=(29.960213627199554%2C%20-98.17086755239256)"

json_raw = requests.get(url).text
json_parsed = json.loads(json_raw)

# Uncomment if needed
# json_parsed

[{'id': 2857,
  'name': 'South Austin: The Range',
  'position_lat': 30.172366,
  'position_lon': -97.780524,
  'address_street': '8301 S IH 35 Frontage Rd',
  'address_city': 'Austin',
  'address_state': 'TX',
  'address_zip': '78745',
  'address_country': 'US',
  'region': 'Austin',
  'session_data': [{'location_session_id': 'LOC-T1563348095LOC-5d2ecc7fe99f8',
    'location_session_name': 'East Austin: The Range - 5:00 AM - T,TH',
    'location_session_desc': 'In Front of The Range',
    'location_session_start_time': '05:00:00',
    'location_session_end_time': '06:00:00',
    'location_session_register_start_date': 'Sun Jul 28 2019 00:00:00 GMT+0000 (UTC)',
    'location_session_register_end_date': None,
    'location_session_visible_start_date': '2019-08-04T00:00:00+00:00',
    'location_session_visible_end_date': None,
    'location_session_checkin_start_date': '2019-08-04T00:00:00+00:00',
    'location_session_checkin_end_date': None,
    'location_session_alternate': {'location

The data is in a complicated json blob; here is the rough schema (with **lots** of columns removed):

```json

[{
    'id': 1234,
    'name': 'Camp Location 1',
    'session_data': [
        {
            'location_session_id': 'loc_sess_1',
            'location_session_start_time': '05:00:00',
            'location_session_end_time': '06:00:00',
            'location_session_trainers': [{
                'location_session_trainer_first_name': 'Tucker',
                'location_session_traner_last_name': 'Max',
                'location_session_trainer_split_percent': 100
            }]
        }
    ]
},{
    'id': 5678,
    'name': 'Camp Location 2',
    'session_data': [
        {
            'location_session_id': 'loc_sess_2',
            'location_session_start_time': '06:00:00',
            'location_session_end_time': '07:00:00',
            'location_session_trainers': [{
                'location_session_trainer_first_name': 'Tyler',
                'location_session_traner_last_name': 'Durden',
                'location_session_trainer_split_percent': 100
            }]
        }
    ]
}]
```

First, we put the parsed data in a DataFrame:

In [60]:
cg_data = pd.DataFrame(json_parsed)
cg_data.set_index('id', inplace=True)

cg_data

,address_city,address_country,address_state,address_street,address_zip,created,distance,modified,name,position_lat,position_lon,region,session_data
id,,,,,,,,,,,,,
2857,Austin,US,TX,8301 S IH 35 Frontage Rd,78745,2018-05-08T23:33:42+00:00,0.5047572665530058,2019-10-04T11:00:55+00:00,South Austin: The Range,30.172366,-97.780524,Austin,[{'location_session_id': 'LOC-T1563348095LOC-5...
5556,Austin,US,TX,7500 S Pleasant Valley Rd,78744,2019-05-13T23:03:15+00:00,2.224177772757142,2019-10-04T11:02:24+00:00,South Austin: Perez Elementary,30.170851,-97.756238,Austin,[{'location_session_id': 'LOC-X1563259472LOC-5...
4415,Austin,US,TX,907 W Slaughter Ln,78748,2018-07-17T11:02:23+00:00,2.8357350893441162,2019-10-04T11:01:41+00:00,South Austin: Mary Moore Searight Metropolitan...,30.162857,-97.808030,Austin,[{'location_session_id': 'LOC-L1566271743LOC-5...
3535,Austin,US,TX,8601 Vertex Blvd,78747,2018-05-08T23:34:33+00:00,2.917957048110134,2019-10-04T11:01:13+00:00,South Austin: Blazier Elementary,30.156114,-97.752070,Austin,[{'location_session_id': 'LOC-P1525988980LOC-5...
2691,Austin,US,TX,10701 S 1st St,78748,2018-05-08T23:33:29+00:00,3.0391408687097843,2019-10-04T11:00:50+00:00,South Austin: Akins High School,30.148984,-97.801855,Austin,[{'location_session_id': 'LOC-I1566351786LOC-5...
969,Austin,US,TX,6405 Circle S Rd,78745,2018-05-08T23:32:23+00:00,3.2339228760221066,2019-10-04T11:00:20+00:00,South Austin: Pleasant Hill Elementary,30.196949,-97.776053,Austin,[{'location_session_id': 'LOC-U1563259613LOC-5...
2289,Austin,US,TX,7601 Dixie Dr,78744,2018-05-08T23:33:14+00:00,3.4273951985410402,2019-10-04T11:00:44+00:00,South Austin: Palm Elementary,30.164824,-97.743673,Austin,[{'location_session_id': 'LOC-I1560494740LOC-5...
4414,Austin,US,None,9400 Texas Oaks Dr,TX,2018-07-17T11:02:23+00:00,3.5633335906028276,2019-10-04T11:01:41+00:00,South Austin: Casey Elementary,30.177268,-97.814616,Austin,[{'location_session_id': 'LOC-H1560405285LOC-5...
2366,Austin,US,TX,5106 Village Square Dr,78744,2018-05-08T23:33:17+00:00,4.161766929061865,2019-10-04T11:00:45+00:00,South Austin: Mendez Middle School,30.188675,-97.743055,Austin,[{'location_session_id': 'LOC-P1498500627LOC-5...


Then, we extract just the `session_data` column, which will be in several joined tables:

In [68]:
cg_sessions = json_normalize(data=json_parsed, record_path='session_data', meta=['id'])
cg_sessions.set_index('location_session_id', inplace=True)

cg_sessions

,location_session_alternate,location_session_bold_days_formated,location_session_bold_days_simple,location_session_checkin_end_date,location_session_checkin_end_date_formatted,location_session_checkin_start_date,location_session_checkin_start_date_formatted,location_session_days,location_session_days_formated,location_session_days_simple,...,location_session_name_alternate,location_session_register_end_date,location_session_register_start_date,location_session_start_time,location_session_start_time_formated,location_session_trainers,location_session_visible_end_date,location_session_visible_start_date,location_session_visible_start_date_formatted,id
location_session_id,,,,,,,,,,,,,,,,,,,,,
LOC-T1563348095LOC-5d2ecc7fe99f8,{'location_alternate_name': 'Awning Under Entr...,,[],None,NaN,2019-08-04T00:00:00+00:00,08/04/19,"[{'location_session_day_name': 'T', 'location_...","T, TH","[T, TH]",...,Awning Under Entrance,None,Sun Jul 28 2019 00:00:00 GMT+0000 (UTC),05:00:00,5:00 AM,[{'location_session_trainer_first_name': 'Mark...,None,2019-08-04T00:00:00+00:00,08/04/19,2857
LOC-K1551333364LOC-5c7777f4281cb,{'location_alternate_name': 'Awning Under Fron...,W,[W],None,NaN,2019-03-17T00:00:00+00:00,03/17/19,"[{'location_session_day_name': 'M', 'location_...","M, W","[M, W]",...,Awning Under Front Entrance,None,Sun Mar 10 2019 00:00:00 GMT+0000 (UTC),05:15:00,5:15 AM,[{'location_session_trainer_first_name': 'Mark...,None,2019-03-17T00:00:00+00:00,03/17/19,2857
LOC-X1563259472LOC-5d2d7250b176d,{'location_alternate_name': 'Basketball Court ...,W,[W],None,NaN,2019-08-04T00:00:00+00:00,08/04/19,"[{'location_session_day_name': 'M', 'location_...","M, W","[M, W]",...,"Basketball Court At 7500 S Pleasant Valley Rd,...",None,Sun Jul 28 2019 00:00:00 GMT+0000 (UTC),17:00:00,5:00 PM,[{'location_session_trainer_first_name': 'Melo...,None,2019-08-04T00:00:00+00:00,08/04/19,5556
LOC-P1563259537LOC-5d2d7291cd705,{'location_alternate_name': 'Basketball Court'},W,[W],None,NaN,2019-08-04T00:00:00+00:00,08/04/19,"[{'location_session_day_name': 'M', 'location_...","M, W","[M, W]",...,Basketball Court,None,Sun Jul 28 2019 00:00:00 GMT+0000 (UTC),18:30:00,6:30 PM,[{'location_session_trainer_first_name': 'Melo...,None,2019-08-04T00:00:00+00:00,08/04/19,5556
LOC-L1566271743LOC-5d5b68ffa8249,{'location_alternate_name': 'Pavillion Near Cr...,TH,[TH],None,NaN,2019-09-08T00:00:00+00:00,09/08/19,"[{'location_session_day_name': 'M', 'location_...","M, W, TH","[M, W, TH]",...,"Pavillion Near Creek, Set Back From Parking Lot",None,Tue Aug 13 2019 00:00:00 GMT+0000 (UTC),08:00:00,8:00 AM,[{'location_session_trainer_first_name': 'Jean...,None,2019-09-08T00:00:00+00:00,09/08/19,4415
LOC-H1566271820LOC-5d5b694cdcd04,{'location_alternate_name': 'Pavillion Near Cr...,TH,[TH],None,NaN,2019-09-08T00:00:00+00:00,09/08/19,"[{'location_session_day_name': 'M', 'location_...","M, W, TH","[M, W, TH]",...,"Pavillion Near Creek, Set Back From Parking Area",None,Tue Aug 13 2019 00:00:00 GMT+0000 (UTC),09:15:00,9:15 AM,[{'location_session_trainer_first_name': 'Jean...,None,2019-09-08T00:00:00+00:00,09/08/19,4415
LOC-P1525988980LOC-5af4be74867e4,{'location_alternate_name': 'Awnings In Front ...,"M, F","[M, F]",2019-10-12T00:00:00+00:00,10/12/19,2018-05-28T00:00:00+00:00,05/28/18,"[{'location_session_day_name': 'M', 'location_...","M, W, F","[M, W, F]",...,Awnings In Front Of School,Sun Oct 06 2019 00:00:00 GMT+0000 (UTC),Sun May 20 2018 00:00:00 GMT+0000 (UTC),05:05:00,5:05 AM,[{'location_session_trainer_first_name': 'Larr...,Sat Oct 12 2019 00:00:00 GMT+0000 (UTC),2018-05-20T00:00:00+00:00,05/20/18,3535
LOC-V1531977544LOC-5b501f4827f92,{'location_alternate_name': 'Awnings In Front ...,S,[S],None,NaN,2018-08-06T00:00:00+00:00,08/06/18,"[{'location_session_day_name': 'S', 'location_...",S,[S],...,Awnings In Front of School,None,Sun Jul 29 2018 00:00:00 GMT+0000 (UTC),07:30:00,7:30 AM,[],None,2018-07-29T00:00:00+00:00,07/29/18,3535
LOC-H1531977343LOC-5b501e7f780ce,{'location_alternate_name

Here, we are flattening the trainer information, so that we can get the trainer(s) for each session. There is probably a way to do this within Pandas, but I couldn't figure it out, and this was pretty easy to write.

In [101]:
dict = cg_sessions.to_dict('dict')
trainers_dict = dict['location_session_trainers']
trainers_list = []

for key in trainers_dict:
    for item in trainers_dict[key]:        
        trainer = copy(item)
        trainer['location_session_id'] = key
        trainers_list.append(trainer)
        
cg_trainers = pd.DataFrame(trainers_list)
cg_trainers

,location_session_id,location_session_trainer_first_name,location_session_trainer_image_url,location_session_trainer_is_primary,location_session_trainer_last_name,location_session_trainer_phone,location_session_trainer_rating,location_session_trainer_split_percent
0,LOC-T1563348095LOC-5d2ecc7fe99f8,Mark,http://cgcdn.s3.amazonaws.com/nation/users/UID...,None,White,(504) 982-2950,4.9,100
1,LOC-K1551333364LOC-5c7777f4281cb,Mark,http://cgcdn.s3.amazonaws.com/nation/users/UID...,None,White,(504) 982-2950,4.9,100
2,LOC-X1563259472LOC-5d2d7250b176d,Melody,https://s3.amazonaws.com/realm-production/user...,None,Yargeau,(860) 942-6609,4.9,100
3,LOC-P1563259537LOC-5d2d7291cd705,Melody,https://s3.amazonaws.com/realm-production/user...,None,Yargeau,(860) 942-6609,4.9,100
4,LOC-L1566271743LOC-5d5b68ffa8249,Jean-Marie,http://cgcdn.s3.amazonaws.com/nation/users/UID...,None,Langlois,(870) 530-7333,4.9,100
5,LOC-H1566271820LOC-5d5b694cdcd04,Jean-Marie,http://cgcdn.s3.amazonaws.com/nation/users/UID...,None,Langlois,(870) 530-7333,4.9,100
6,LOC-P1525988980LOC-5af4be74867e4,Larry,http://cgcdn.s3.amazonaws.com/nation/users/UID...,None,John II,(512) 665-7676,4.9,100
7,LOC-S1560491291LOC-5d03351bb526a,Ashlyn,https://s3.amazonaws.com/realm-production/user...,None,Harmon,(214) 695-0483,4.9,100
8,LOC-O1560491331LOC-5d033543c07bb,Ashlyn,https://s3.amazonaws.com/realm-production/user...,None,Harmon,(214) 695-0483,4.9,100
9,LOC-I1566351786LOC-5d5ca1aae6803,Jeff,http://cgcdn.s3.amazonaws.com/nation/users/UID...,None,Ward,(361) 772-2271,4.9,100


Here, we are merging the tables together via a join.

In [ ]:
cg_session_trainers = pd.merge(cg_sessions, cg_trainers, on='location_session_id')
cg_session_trainers_data = pd.merge(cg_data, cg_session_trainers, on='id')

And finally, we export the data to Excel. 

In [103]:
cg_data_export = cg_session_trainers_data[['id', 'address_street', 'address_city', 'address_state', 'address_zip', 'created', 'modified', 'name', 'position_lat', 'position_lon', 'region', 'location_session_bold_days_simple', 'location_session_checkin_start_date_formatted', 'location_session_checkin_end_date_formatted', 'location_session_days_simple', 'location_session_desc', 'location_session_end_time', 'location_session_name', 'location_session_name_alternate', 'location_session_start_time', 'location_session_visible_start_date', 'location_session_visible_end_date', 'location_session_visible_start_date_formatted', 'location_session_trainer_first_name', 'location_session_trainer_last_name', 'location_session_trainer_image_url', 'location_session_trainer_is_primary', 'location_session_trainer_phone', 'location_session_trainer_rating', 'location_session_trainer_split_percent']]
cg_data_export.to_excel('cg_session_data.xlsx')